In [1]:
import pyvisa

In [2]:
rm = pyvisa.ResourceManager()
rm.list_resources()

('ASRL8::INSTR', 'ASRL13::INSTR')

In [94]:
a = rm.open_resource('ASRL13::INSTR')

In [95]:
def int_to_bytes(i):
    return (i).to_bytes(1, byteorder='big')

In [96]:
def read_registers(arduino):
    print(arduino.query('r'), end='')
    for i in range(23):
        print(arduino.read(), end='')

In [97]:
read_registers(a)

Reading register contents:
   0: 00000000,00000000,00000000,00000000,
   1: 00000000,01000000,00001000,00100000,
   2: 00011111,00111111,01000000,00000000,
   3: 00000000,00000000,01111111,01111111,
   4: 11111111,11111111,11111111,11111111,
   5: 01101001,00100000,01000000,10000001,01000100,01100000,
   6: 00000011,00100000,00000001,10001001,00000000,00100000,
   7: 00000000,00000000,00000000,00000000,
   8: 00000000,00000000,
   9: 00000000,00000000,00000000,00000000,
   A: 00000000,00000000,00000000,00000000,
   B: 00000000,10010001,00000100,00011000,00011001,00100010,00000010,11000000,
   C: 01100100,10000000,10010000,00000001,00000011,00000000,10000010,10000000,
   D: 01000000,11010000,00000000,00001000,
   E: 00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,
   F: 00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,
  10: 00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,
  11: 00000000,00000000,00000000,00000000,00

In [75]:
def write_register(arduino, reg, data_list):
    # send the write command
    arduino.write_raw(
        b"w" \
          + bytes(bytearray([reg])) \
          + bytes(bytearray(data_list))
      )
    
    # print Arduino's return text
    print(arduino.read(), end='')
    print(arduino.read(), end='')

In [24]:
def reset(arduino):
    print(arduino.query('0'))

In [124]:
reset(a)

Device reset complete.



In [125]:
write_register(a, 0x0, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x1, [0b00000000,0b01000000,0b00001000,0b00100000,])
write_register(a, 0x2, [0b00011111,0b00111111,0b11000000,0b00000000,])
write_register(a, 0x3, [0b00000000,0b00000000,0b01111111,0b01111111,])
write_register(a, 0x4, [0b11111111,0b11111111,0b11111111,0b11111111,])
write_register(a, 0x5, [0b11101001,0b00100000,0b01100100,0b10001001,0b11000100,0b11100000,])
write_register(a, 0x6, [0b10000111,0b00101100,0b00000101,0b10101001,0b10000000,0b00101001,])
write_register(a, 0x7, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x8, [0b00000000,0b00000000,])
write_register(a, 0x9, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0xA, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0xB, [0b10100001,0b10011001,0b11000100,0b00011010,0b00011001,0b00101110,0b00000011,0b11000100,])
write_register(a, 0xC, [0b11000100,0b10101001,0b10110000,0b10000101,0b00000011,0b11000000,0b10000011,0b10110000,])
write_register(a, 0xD, [0b01001110,0b11010000,0b00000000,0b00011100,])
write_register(a, 0xE, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0xF, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x10, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x11, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x12, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x13, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x14, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x15, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x16, [0b00111000,0b11010000,0b01010011,0b11000111,])

Writing to register 0:   
  00000000,00000000,00000000,00000000,
Writing to register 1:   
  00000000,01000000,00001000,00100000,
Writing to register 2:   
  00011111,00111111,11000000,00000000,
Writing to register 3:   
  00000000,00000000,01111111,01111111,
Writing to register 4:   
  11111111,11111111,11111111,11111111,
Writing to register 5:   
  11101001,00100000,01100100,10001001,11000100,11100000,
Writing to register 6:   
  10000111,00101100,00000101,10101001,10000000,00101001,
Writing to register 7:   
  00000000,00000000,00000000,00000000,
Writing to register 8:   
  00000000,00000000,
Writing to register 9:   
  00000000,00000000,00000000,00000000,
Writing to register A:   
  00000000,00000000,00000000,00000000,
Writing to register B:   
  10100001,10011001,11000100,00011010,00011001,00101110,00000011,11000100,
Writing to register C:   
  11000100,10101001,10110000,10000101,00000011,11000000,10000011,10110000,
Writing to register D:   
  01001110,11010000,00000000,00011100,


In [93]:
a.close()